In [3]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

In [4]:
articles = pd.read_json('data/articles.json.xz')
google_qa = pd.read_json('data/google_qa.json.xz')
jokes = pd.read_json('data/jokes.json.xz')

In [103]:
jokes_and_not = {'text': [], 'funny': []}

counter = 0
for line in articles.text[:10000]:
    jokes_and_not['text'].append(line)
    jokes_and_not['funny'].append(0)

for line in google_qa.text[:10000]:
    jokes_and_not['text'].append(line)
    jokes_and_not['funny'].append(0)

for line in jokes.text[:20000]:
    jokes_and_not['text'].append(line)
    jokes_and_not['funny'].append(1)

In [104]:
df = pd.DataFrame(jokes_and_not)

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['funny'], test_size = 0.1)

In [106]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv1D
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Embedding

In [107]:
tokenizer = Tokenizer(num_words=4096)

tokenizer.fit_on_texts(X_train)

tokenized_X_train = tokenizer.texts_to_sequences(X_train)
tokenized_X_test = tokenizer.texts_to_sequences(X_test)

maxlen=20

x_train_pad = preprocessing.sequence.pad_sequences(tokenized_X_train, maxlen=maxlen)
x_test_pad = preprocessing.sequence.pad_sequences(tokenized_X_test, maxlen=maxlen)

In [110]:

model = Sequential()

model.add(Embedding(len(tokenizer.word_index), 8, input_length=maxlen))

model.add(Conv1D(256, 10, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))

In [111]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train_pad, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

model.evaluate(x_test_pad, y_test)

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 20, 8)             1007456   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 11, 256)           20736     
_________________________________________________________________
flatten_9 (Flatten)          (None, 2816)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 10)                28170     
_________________________________________________________________
flatten_10 (Flatten)         (None, 10)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 11        
Total params: 1,056,373
Trainable params: 1,056,373
Non-trainable params: 0
___________________________________________

[0.1341034471988678, 0.9637500047683716]

In [117]:
def predict(nali):
    nali2 = tokenizer.texts_to_sequences(nali)
    print(nali2)
    nali3 = preprocessing.sequence.pad_sequences(nali2, maxlen=maxlen)
    s = model.predict(nali3)
    return s


predict("What do you call hitler?")


[[178], [1221], [3], [730], [], [682], [877], [], [1045], [877], [54], [], [385], [3], [1335], [1335], [], [1221], [17], [730], [1335], [277], [758], []]


array([[0.37984994],
       [0.33767396],
       [0.7726667 ],
       [0.15749133],
       [0.04854256],
       [0.33619687],
       [0.30800676],
       [0.04854256],
       [0.90622085],
       [0.30800676],
       [0.13772249],
       [0.04854256],
       [0.50066406],
       [0.7726667 ],
       [0.09551316],
       [0.09551316],
       [0.04854256],
       [0.33767396],
       [0.8980785 ],
       [0.15749133],
       [0.09551316],
       [0.66204065],
       [0.40701824],
       [0.04854256]], dtype=float32)